# Un exemple simple

## Descente de gradient ordinaire

In [1]:
# création d'un jeu d'entrainement
import numpy as np
x=np.array([[0, 0,1], 
            [2, 2,4], 
            [1, 4,1],
            [10, 3,0],
            [1,1,1]])
y=np.array([15.5,
            30,
            25.5,
           27,
           17.5])

In [2]:
from sklearn import linear_model
reg = linear_model.LinearRegression() # création d'un objet LinearRegression
reg.fit(x, y) # apprentissage avec la méthode fit

LinearRegression()

In [3]:
# affichage de l'intercept theta_0
reg.intercept_

12.13296672306825

In [4]:
# affichage de des coefficients theta_j avec j>1
reg.coef_

array([0.76128032, 2.40073322, 2.88141568])

In [5]:
# predictions du modèles sur données d'entrainement
y_hat=reg.predict(x)
y_hat

array([15.0143824 , 29.98265651, 25.3785956 , 26.94796954, 18.17639594])

In [6]:
# Calcul de la MSE sur le jeu d'entrainement
from sklearn.metrics import mean_squared_error
mean_squared_error(y, y_hat)

0.14221658206429777

In [9]:
# création d'un jeu de test
import numpy as np
x_test=np.array([[3, 5,7], 
                 [7, 0,8]])
y_test=np.array([48,
                39])

In [10]:
# predictions du modèles sur données de test
y_hat_test=reg.predict(x_test)
y_hat_test

array([46.59038353, 40.51325437])

In [11]:
# Calcul de la MSE sur le jeu de test
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_hat_test)

2.1384786910666325

## Descente de gradient stochastique

In [12]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(fit_intercept=True,learning_rate="constant",eta0=0.001,max_iter=100000)
reg.fit(x,y)

SGDRegressor(eta0=0.001, learning_rate='constant', max_iter=100000)

In [13]:
reg.intercept_

array([9.89650841])

In [14]:
reg.coef_

array([0.86041433, 2.80755161, 3.39858884])

In [15]:
y_hat=reg.predict(x)
y_hat

array([13.29509725, 30.82679563, 25.38571799, 26.9233065 , 16.96306318])

In [16]:
# Calcul de la MSE sur le jeu d'entrainement
mean_squared_error(y, y_hat)

1.1704861173061283

In [17]:
# predictions du modèles sur données de test
y_hat_test=reg.predict(x_test)
y_hat_test

array([50.30563129, 43.10811941])

In [18]:
# Calcul de la MSE sur le jeu de test
mean_squared_error(y_test, y_hat_test)

11.096290379313016

# Un exemple plus complet à partir de données importées
Dans cet exemple, certaines parties sont à compléter en se référant à la documentation officielle : https://scikit-learn.org/0.21/modules/linear_model.html <br>
Dans cet exemple, l'objectif est de prédire la valeur moyenne des maisons dans les districts californiens en utilisant un modèle de régression linéaire puis polynomial.<br>
Pour atteindre cet objectif, nous suivrons un processus standard qui se décline en 8 grandes étapes :<br>
- Importation et préparation des données avec Pandas
- Création d'un jeu de donnée d'entrainement et de test
- Création d'un pipeline de transformation applicable aux données d'entrainement et de test
- Instanciation et paramètrage d'un modèle de prédiction
- Entrainement du modèle à partir du jeu d'entrainement
- Validation du modèle à partir du jeu d'entrainement
- Performance du modèle mesurée sur jeu de test
- Prediction sur de nouvelle données

Les deux dernières étapes ne sont envisagées que lorsque le modèle a été validé. Une mauvaise performance sur le jeu de test pour nous conduire à changer les valeurs d'hyperparamètres ou a choisir un autre modèle plus ou moins complexe. Ces ajustements trahissent une forme d'apprentissage sur les données de test que nous ne sommes pas censés connaître.

## Importation et préparation des données avec Python

In [8]:
import pandas as pd
data_dir="Dataset/" # chemin relatif vers le fichier de données. A adapter selon l'emplacement choisi pour le fichier de données.
data=pd.read_csv(data_dir+"California_Housing - Copie.csv",delimiter=";")

In [9]:
# Nettoyage des données
data_cleaned=data.loc[data.population<=5*data.population.std()] # suppression des données aberrantes
data_cleaned=data_cleaned.drop("total_pools",axis=1) # suppression de la colonne "total_pools" constituée à 50% de valeurs nulles
data_cleaned=data_cleaned.drop(["longitude","latitude"],axis=1) # suppression des colonnes longitude et latitude
data_cleaned=data_cleaned.dropna(axis=0) # supression des lignes contenant au moins une valeur nulle
data_cleaned=data_cleaned.astype({'total_rooms': 'float64','ocean_proximity':'category'}) # conversion de type

In [10]:
data_cleaned

,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...
20635,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


## Création d'un jeu de donnée d'entrainement et de test

In [18]:
# Utilisez la fonction train_test_split pour créer un jeu de donnée d'entrainement et de test à partir du dataframe data_cleaned
# Faites cette répartition en mettant de côté 20% des données pour la phase de test
from sklearn.model_selection import train_test_split
#Complétez la ligne de code ci-dessous
Train_set, Test_set= train_test_split(data_cleaned,test_size=0.2,random_state=42)

In [19]:
# Il faut maintenant séparer les features des labels
# Utilisez les liste features et label pour faire cette séparation
features=["housing_median_age","total_rooms","total_bedrooms","population","households","median_income","ocean_proximity"]
label=["median_house_value"]

In [20]:
#Complétez le code ci-dessous
Train_set_features=
Test_set_features=
Train_set_label=
Test_set_label=

SyntaxError: invalid syntax (<ipython-input-20-dd9d47fa00ca>, line 2)

## Création d'un pipeline de transformation applicable aux données d'entrainement et de test
Avant de prodécder à l'apprentissage, il est souvent nécessaire de transformer certaines features pour les rendre compatibles avec les logiques propres à chaque algorithme et le type de tâche à réaliser (régression ou classification).<br>
- Quel que soit le modèle d'apprentissage utilisé et la tâche à réaliser, il est nécessaire de fournir aux algorithmes des données d'entrée numériques. Dans notre cas, la variable "ocean_proximity" doit être transformée en valeurs numérique via le transformateur OrdinalEncoder (documentation : https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html) <br>
- La transformation OrdinalEncoder appliquée à  "ocean_proximity" génère une nouvelle veriable  categorielle numérique. Dans le cas d'une tâche de régression, une nouvelle transformation de type OneHotEncoder est nécessaire pour ne pas que le choix des valeurs associées aux variables catégorielles influencent la recherche des paramètres du modèle (documentation : https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)<br>
- Une dernière transformation consiste à normaliser les varaibles numériques continues pour évacuer l'effet que pourraient avoir des echelles ou des ordres de grandeurs hétérogènes. Cette transformation se fait via StandardScaler (documentation : https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)<br>

Toutes ces transformations (OrdinalEncoder,OneHotEncoder et StandardScaler) peuvent être combinées dans un pipeline de type ColumnTransformer (documentation: https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) qui une fois appliqué aux données réalise les transformation dans l'ordre d'apparatition. Un paramètre permet de préciser à chaque fois quelles sont les colonnes qui doivent subit la transformation.

**Remarque:** La transformation OneHotEncoder peut encoder des variables non numériques. Il n'est donc pas nécessaire de faire précéder cette transformation par OrdinalEncoder.

In [30]:
# importation des classes utiles à la transformation et à la création d'un pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler 
from sklearn.compose import ColumnTransformer

In [31]:
# OrdinalEncoder appliqués à la colonne ocean_proximity
Ord_Enc=OrdinalEncoder()
Ord_Enc.fit(Train_set_features[["ocean_proximity"]])
ocean_proximity_Ord_Enc= Ord_Enc.transform(Train_set_features[["ocean_proximity"]])

# OneHotEncoder appliqués à la colonne ocean_proximity_Ord_Enc
Oh_Enc=OneHotEncoder()
Oh_Enc.fit(ocean_proximity_Ord_Enc)
ocean_proximity_Oh_Enc=Oh_Enc.transform(ocean_proximity_Ord_Enc)

In [405]:
# Affichez le contenu de ocean_proximity_Ord_Enc
# Quelle interprétation faites-vous des valeurs affichées ?


In [406]:
# Affichez le contenu de ocean_proximity_Oh_Enc
# Quelle interprétation faites-vous des valeurs affichées ?


In [32]:
# Pour appliquer des tranformations sur des groupes de colonnes on peut construire un pipeline de transformations avec un objet ColumnTransformer
# Nous commençons par distinguer les variables selon leur type (categorielles ou continues)
cat_attribs=["ocean_proximity"]
num_attribs=["housing_median_age","total_rooms","total_bedrooms","population","households","median_income"]

In [157]:
# ColumnTransformer prend comme attribut un liste de tuple chacun correspondant à une transformation particulière
# L'ordre des transformations est importan
# Un tuple représentant une transformation prend la for ("nom_transformation",transformateur(),liste_colonne)
pipeline=ColumnTransformer([
    ("Hot_encoder", OneHotEncoder(),cat_attribs),
    ("Scaler", StandardScaler(),num_attribs)])

In [158]:
# Pour appliquer le pipeline, il suffit d'invoquer la méthode fit_transform
Train_set_features_transformed=pipeline.fit_transform(Train_set_features)  
Test_set_features_transformed=pipeline.fit_transform(Test_set_features) 

## Instanciation et paramètrage d'un modèle de prédiction

In [35]:
from sklearn.linear_model import LinearRegression

In [202]:
# instanciez un modèle de regression linéaire  simple Lin_reg
Lin_reg=

LinearRegression()

**Question:** Quelles sont la classes qui permettent de créer des modèles linéaires résularisés ?<br>
**Remarque:** Le modèle peut être inégré en dernière étape du pipeline

## Entrainement du modèle à partir du jeu d'entrainement
L'ntrainement du modèle peut se faire simplement via la méthode fit.

In [37]:
# Entrainez le modèle Lin_reg en utilisant le jeu d'entrainement


In [39]:
# Affichez les paramètres (coefficients) du modèle appris


In [41]:
# Affichez la performance RMSE du modèle mesurée sur le jeu d'entrainement en utilisant la métrique mean_squared_error et la méthode predict


**Questions :** <br>
- Peut-on savoir si les hyperparamètres sont bons ou pas ?
- Peut-on dire que l'hypothèse linéaire est suffisante ?
- Peut-on savoir si le modèle sur-apprend ?

## Validation du modèle à partir du jeu d'entrainement
Pour mesurer la qualité d'un prédicteur et la capcité de généralisation d'un modèle, il est recommandé d'utiliser une partie du jeu d'entrainement pour calculer les paramètres du modèle et une autre partie pour en mesurer la performance.
En cas de sur ou sous apprentissage, on peut ainsi recalibrer les hyper-paramètres du modèle sans avoir à utiliser le jeu de test.
Pour maximiser l'utilisation des données d'apprentissage pour le calcul des paramètres, une bonne pratique consiste à réaliser une cross-validation par la méthode des K-folds.<br>
Consultez la documentation pour comprendre le principe de cette méthode : https://scikit-learn.org/stable/modules/cross_validation.html

In [43]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(Lin_reg,Train_set_features_transformed,Train_set_label, cv=3,scoring='neg_mean_squared_error')

In [430]:
# Affichez le contenu de scores. Que représentent les valeurs qui s'affichent ?


In [45]:
# Représentation sur un graphe 
from sklearn.model_selection import learning_curve
train_sizes, train_scores, valid_scores = learning_curve(Lin_reg, Train_set_features_transformed, Train_set_label,train_sizes=list(range(100,9700,100)), cv=3,scoring='neg_mean_squared_error')

**Question:** Analysez rapidement le contenu de train_sizes, train_scores et valid_scores 

In [46]:
import matplotlib.pyplot as plt

def Train_validation_curves(train_sizes,train_scores,valid_scores):
    t = train_sizes
    fig, ax1 = plt.subplots()

    color='tab:red'
    ax1.set_xlabel('Training Sample Size')
    ax1.set_ylabel('RMSE Train',color=color)
    ax1.plot(t, np.mean(np.sqrt(-train_scores),axis=1), color=color,label='Train')
    ax1.tick_params(axis='y', labelcolor=color)

    color='tab:blue'
    ax2 = ax1.twinx()
    ax2.set_ylabel('RMSE Validation',color=color)
    ax2.plot(t, np.mean(np.sqrt(-valid_scores),axis=1),color=color ,label='Validation')
    ax2.tick_params(axis='y', labelcolor=color)

    plt.show()

In [450]:
# Utilisez la fonction Train_validation_curves pour afficher les courbes d'apprentissage et de validation


**Question:** Que représentent les points qui ont permis de tracer les courbes bleue et rouge ? (analysez le code de la fonction Train_validation_curves).

## Performance du modèle mesurée sur jeu de test
**Questions:**
- Si nous considérons que le modèle linéaire est satisfaisant, quelle est la méthode à utiliser pour en mesurer la performance sur le jeu de test ?
- Compte tenu des résultats de validation, le modèle linéaire vous semble-t-il satisfaisant ?
- Quel phénomène observez vous en analysant les courbes d'apprentissage et de validation ?
- Est-il raisonnable de tester ce modèle ?

## Prédictions sur de nouvelles données
**Questions:**
- Si nous considérons que le modèle linéaire est satisfaisant, quelle est la méthode à utiliser pour faire des prédictions sur de nouvelles données ?
- Quelles sont les étapes de préparation à prévoir si ces nouvelles données se présentent sous la même forme que les données d'entrainement et de test ?

# Régression polynomiale
Un modèle polynomial n'est rien d'autres qu'un modèle linéaire entrainé sur les données d'origines auquelles on ajoutes de nouvelles variables calaculées à partir des variables initiales.
Nous allons créer un nouveau pipeline qui introduit l'ajout de ces nouvelles variables.

In [48]:
# Créez un nouveau pipeline qui intégre une étape d'enrichissement des features
# avec de nouvelles varaibles "polynoamiales"
# utilisez le transformateur PolynomialFeatures (documentation : https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)

from sklearn.preprocessing import PolynomialFeatures

pipeline=ColumnTransformer([
    ("Hot_encoder", OneHotEncoder(),cat_attribs),
    ("Scaler", StandardScaler(),num_attribs),
    # complétez le code ici
])

In [150]:
# Appliquer le pipeline à Train_set_features et Test_set_features
Train_set_features_transformed=
Test_set_features_transformed=

In [151]:
# Comparez les dimensions de Train_set_features_transformed et de Train_set_features
# Que remarquez-vous ?


In [152]:
# Entrainez votre modèle en utilisant une regression linéaire régularisée (lasso) avec descente de gradient stochastique


In [154]:
# Affichez la Performance (RMSE) du modèle sur données d'entrainement


RMSE:  8.517923820363152e+22


In [155]:
# Affichez les coefficients du modèle entrainé
